In [28]:
# Load libraries
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [32]:
data = pd.read_csv("dataFiltrada.csv", low_memory = False)
noPrice = pd.read_csv("properati_dataset_testing_noprice.csv", low_memory = False)

In [104]:
#Solo me quedo con CAP FED y GBA
dataCapGba = data.loc[(data.state_name.str.contains('Capital Federal') | data.state_name.str.contains('G.B.A') ), :]
dataCapGba = dataCapGba.loc[dataCapGba['price_usd_per_m2'] < 9000,:]
dataCapGba = dataCapGba.loc[dataCapGba['price_usd_per_m2'] > 1000,:]
dataCapGba = dataCapGba.loc[dataCapGba['surface_total_in_m2'] > 5, :]

In [105]:
#Elimino duplicados y registros con ALGUNA COLUMNA NaN (no nos serviran para predecir)
dataCapGba.drop_duplicates(subset=['lat', 'lon', 'price_usd_per_m2'])
dataCapGba.dropna(inplace=True, how='any')
dataCapGba.head(20)

,Unnamed: 0,index,created_on,operation,property_type,state_name,place_name,lat,lon,price,...,cant_prop,media_por_barrio,mediana_por_barrio,geoPos,distancia_al_centro,distanciaAIndoamericano,distanciaAFuerteApache,distanciaACarlosGardel,cant_subtes_cercanos,cant_trenes_cercanos
1,1,2,2014-03-25,sell,house,Bs.As. G.B.A. Zona Norte,Pilar,-34.440752,-58.845949,220000.0,...,1004.0,1450.725589,1445.113788,"(-34.440752000399996, -58.845949173000001)",47404.183107,42653.282345,34566.588870,31958.058490,0,0
2,2,3,2014-03-25,sell,house,Bs.As. G.B.A. Zona Norte,Pilar,-34.426169,-58.780289,210000.0,...,1004.0,1450.725589,1445.113788,"(-34.426168904400001, -58.780288696299998)",42683.661730,39028.318594,31012.749860,28822.768850,0,0
3,3,4,2014-03-25,sell,house,Bs.As. G.B.A. Zona Norte,Pilar,-34.470052,-58.971004,590000.0,...,1004.0,1450.725589,1445.113788,"(-34.470052078800002, -58.9710044861)",57150.478593,50893.403467,42981.839727,39911.653815,0,0
13,13,15,2014-03-25,sell,apartment,Bs.As. G.B.A. Zona Norte,Pilar,-34.455906,-58.908329,97000.0,...,1004.0,1450.725589,1445.113788,"(-34.455905899999998, -58.908329000000002)",52134.639818,46536.173643,38506.133573,35619.815135,0,0
14,14,16,2014-03-25,sell,apartment,Bs.As. G.B.A. Zona Norte,Pilar,-34.455906,-58.908329,110000.0,...,1004.0,1450.725589,1445.113788,"(-34.455905899999998, -58.908329000000002)",52134.639818,46536.173643,38506.133573,35619.815135,0,0
16,16,18,2014-03-25,sell,house,Bs.As. G.B.A. Zona Norte,Pilar,-34.447366,-58.953538,350000.0,...,1004.0,1450.725589,1445.113788,"(-34.447366199999998, -58.953537699999998)",56366.206490,50615.022219,42612.806051,39667.809009,0,0
17,17,19,2014-03-25,sell,house,Bs.As. G.B.A. Zona Norte,Pilar,-34.456783,-58.905406,295000.0,...,1004.0,1450.725589,1445.113788,"(-34.456782799999999, -58.905405799999997)",51849.336392,46255.331782,38224.224055,35340.609024,0,0
18,18,20,2014-03-25,sell,house,Bs.As. G.B.A. Zona Norte,Pilar,-34.430463,-58.775192,260000.0,...,1004.0,1450.725589,1445.113788,"(-34.430463099999997, -58.775191599999999)",42045.446500,38360.445651,30344.259993,28156.198028,0,0
19,19,21,2014-03-25,sell,house,Bs.As. G.B.A. Zona Norte,Pilar,-34.447366,-58.953538,350000.0,...,1004.0,1450.725589,1445.113788,"(-34.447366199999998, -58.953537699999998)",56366.206490,50615.022219,42612.806051,39667.809009,0,0
20,20,22,2014-03-25,sell,house,Bs.As. G.B.A. Zona Norte,Pilar,-34.455906,-58.908329,90000.0,...,1004.0,1450.725589,1445.113788,"(-34.455905899999998, -58.908329000000002)",52134.639818,46536.173643,38506.133573,35619.815135,0,0


In [106]:
#Transformemos las columnas created_on y property_type a valores numericos para poder usarlos a la hora de predecir
#Para ello, crearemos algunas funciones diccionario que nos sirvan para dicha informacion:

#property_type's: apartament, house, PH, store
def tipoDePropiedad(x):
    tipos = {
        "apartment" : 1,
        "house" : 2,
        "PH" : 3,
        "store" : 4
    }
    return tipos[x] if x in tipos else x

from datetime import datetime
#created_on's: yyyy-mm-dd
def fechaNumerica(x):
    d = datetime.strptime('2012-02-10' , '%Y-%m-%d')
    return int("%4d%02d%02d" % (d.year, d.month, d.day)) + 0

In [107]:
#Aplicamos las funciones anteriormente creadas
dataCapGba.loc[:, 'property_type'] = dataCapGba.property_type.apply(tipoDePropiedad)
dataCapGba.loc[:, 'created_on'] = dataCapGba.created_on.apply(fechaNumerica)

In [111]:
#Ahora, todas las columnas son numericas
dataCapGba.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38037 entries, 1 to 46002
Data columns (total 22 columns):
Unnamed: 0                 38037 non-null int64
index                      38037 non-null int64
created_on                 38037 non-null int64
operation                  38037 non-null object
property_type              38037 non-null int64
state_name                 38037 non-null object
place_name                 38037 non-null object
lat                        38037 non-null float64
lon                        38037 non-null float64
price                      38037 non-null float64
surface_total_in_m2        38037 non-null float64
price_usd_per_m2           38037 non-null float64
cant_prop                  38037 non-null float64
media_por_barrio           38037 non-null float64
mediana_por_barrio         38037 non-null float64
geoPos                     38037 non-null object
distancia_al_centro        38037 non-null float64
distanciaAIndoamericano    38037 non-null float64
dis

In [85]:
#Convertimos todo a int
testData = dataCapGba
testData.loc[:, 'surface_total_in_m2'] = testData.surface_total_in_m2.apply(lambda x: int(x))
testData.loc[:, 'distancia_al_centro'] = testData.distancia_al_centro.apply(lambda x: int(x))
testData.loc[:, 'distanciaAIndoamericano'] = testData.distanciaAIndoamericano.apply(lambda x: int(x))
testData.loc[:, 'distanciaAFuerteApache'] = testData.distanciaAFuerteApache.apply(lambda x: int(x))
testData.loc[:, 'distanciaACarlosGardel'] = testData.distanciaACarlosGardel.apply(lambda x: int(x))
testData.loc[:, 'lat'] = testData.lat.apply(lambda x: round(x,2))
testData.loc[:, 'lon'] = testData.lon.apply(lambda x: round(x,2))

In [86]:
#Me quedo con las columnas que me parecen interesantes y voy a usar para entrenar al modelo.
#testData = testData.loc[:,['created_on', 'property_type', 'lat', 'lon', 'surface_total_in_m2', 'distancia_al_centro', 'cant_subtes_cercanos', 'cant_trenes_cercanos', 'price']] 
#KNN: 0.751849 (0.006051)
#CART: 0.886424 (0.006321)
#NB: 0.886588 (0.005509)
testData = testData.loc[:,['created_on', 'property_type', 'lat', 'lon', 'surface_total_in_m2', 'distancia_al_centro', 'distanciaAIndoamericano', 'distanciaAFuerteApache', 'distanciaACarlosGardel', 'price']] 
#KNN: 0.628447 (0.006205)
#CART: 0.884452 (0.007953)
#NB: 0.877650 (0.005767)
#testData = testData.loc[:,['created_on', 'property_type', 'lat', 'lon', 'surface_total_in_m2', 'distancia_al_centro', 'distanciaAIndoamericano', 'distanciaAFuerteApache', 'distanciaACarlosGardel', 'cant_subtes_cercanos', 'cant_trenes_cercanos', 'price']] 
#KNN: 0.628447 (0.006205)
#CART: 0.883861 (0.007171)
#NB: 0.877650 (0.005767)
#testData = testData.loc[:,['created_on', 'property_type', 'lat', 'lon', 'surface_total_in_m2', 'distancia_al_centro', 'distanciaAIndoamericano', 'cant_subtes_cercanos', 'price']] 
#KNN: 0.678038 (0.004606)
#CART: 0.885701 (0.005193)
#NB: 0.883039 (0.004738)
#testData = testData.loc[:,['created_on', 'property_type', 'lat', 'lon', 'surface_total_in_m2', 'distancia_al_centro', 'price']]
#KNN: 0.751849 (0.006051)
#CART: 0.888034 (0.007477)
#NB: 0.886588 (0.005509)
#testData = testData.loc[:,['created_on', 'property_type', 'lat', 'lon', 'surface_total_in_m2', 'price']]
#KNN: 0.867593 (0.005700)
#CART: 0.889283 (0.006564)
#NB: 0.901968 (0.005843)

In [87]:
testData.head(20)

,created_on,property_type,lat,lon,surface_total_in_m2,distancia_al_centro,distanciaAIndoamericano,distanciaAFuerteApache,distanciaACarlosGardel,price
1,20120210,2,-34.44,-58.85,160,47404,42653,34566,31958,220000.0
2,20120210,2,-34.43,-58.78,180,42683,39028,31012,28822,210000.0
3,20120210,2,-34.47,-58.97,473,57150,50893,42981,39911,590000.0
13,20120210,1,-34.46,-58.91,55,52134,46536,38506,35619,97000.0
14,20120210,1,-34.46,-58.91,54,52134,46536,38506,35619,110000.0
16,20120210,2,-34.45,-58.95,300,56366,50615,42612,39667,350000.0
17,20120210,2,-34.46,-58.91,240,51849,46255,38224,35340,295000.0
18,20120210,2,-34.43,-58.78,180,42045,38360,30344,28156,260000.0
19,20120210,2,-34.45,-58.95,240,56366,50615,42612,39667,350000.0
20,20120210,2,-34.46,-58.91,85,52134,46536,38506,35619,90000.0


In [89]:
# Split-out validation dataset
array = testData.values
X = array[:,0:9]
Y = array[:,9]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [90]:
# Evaluando distintos algoritmos..
SCORING = 'accuracy'

aModelos = []
#aModelos.append(('LR', LogisticRegression()))
aModelos.append(('LDA', LinearDiscriminantAnalysis()))
aModelos.append(('KNN', KNeighborsClassifier()))
aModelos.append(('CART', DecisionTreeClassifier()))
aModelos.append(('NB', GaussianNB()))
#aModelos.append(('SVM', SVC()))

aResultados = []
aNombres = []
for nombre, modelo in aModelos:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    res = model_selection.cross_val_score(modelo, X_train, Y_train, cv=kfold, scoring=SCORING)
    aResultados.append(res)
    aNombres.append(nombre)
    print "%s: %s %f (%f)" % (nombre, SCORING, res.mean(), res.std())

LDA: accuracy 0.024023 (0.002166)
KNN: accuracy 0.075224 (0.003755)
CART: accuracy 0.148444 (0.004906)
NB: accuracy 0.022117 (0.002068)


## Aca Arranca

In [33]:
concatenado = pd.concat([data, noPrice], axis=0, ignore_index=True)
concatenado.dropna(inplace=True, subset=['surface_covered_in_m2', 'surface_total_in_m2'], how='all')
concatenado = concatenado.loc[:,['created_on', 'property_type', 'place_name', 'lat', 'lon', 'surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms', 'description']]
concatenado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59395 entries, 0 to 60168
Data columns (total 10 columns):
created_on               59395 non-null object
property_type            59395 non-null object
place_name               59395 non-null object
lat                      55977 non-null float64
lon                      55977 non-null float64
surface_total_in_m2      57856 non-null float64
surface_covered_in_m2    13005 non-null float64
floor                    1355 non-null float64
rooms                    7342 non-null float64
description              13392 non-null object
dtypes: float64(6), object(4)
memory usage: 5.0+ MB


In [383]:
#def llenadoDeSuperficieTotal(reg):
#    created_on, property_type, place_name, lat, lon, surface_total_in_m2, surface_covered_in_m2, floor, rooms, description = reg
 #   
 #   if np.isnan(surface_total_in_m2):
 #       return surface_covered_in_m2
 #   else:
 #       return surface_total_in_m2
 #   

In [36]:
concatenado['surface_covered_in_m2'] = np.where(np.isnan(concatenado['surface_covered_in_m2']), concatenado['surface_total_in_m2'], concatenado['surface_covered_in_m2'])
concatenado['surface_total_in_m2'] = np.where(np.isnan(concatenado['surface_total_in_m2']), concatenado['surface_covered_in_m2'], concatenado['surface_total_in_m2'])
concatenado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59395 entries, 0 to 60168
Data columns (total 10 columns):
created_on               59395 non-null object
property_type            59395 non-null object
place_name               59395 non-null object
lat                      55977 non-null float64
lon                      55977 non-null float64
surface_total_in_m2      59395 non-null float64
surface_covered_in_m2    59395 non-null float64
floor                    1355 non-null float64
rooms                    7342 non-null float64
description              13392 non-null object
dtypes: float64(6), object(4)
memory usage: 5.0+ MB


In [87]:
superficie_total_barrio = concatenado.loc[:, ['place_name', 'surface_total_in_m2']]
superficie_total_barrio = superficie_total_barrio.groupby('place_name').agg(np.mean)
dicc_superficie_total = {}
for x in xrange(len(superficie_total_barrio)):
    dicc_superficie_total[superficie_total_barrio.index[x]] = superficie_total_barrio.surface_total_in_m2[x]
dicc_superficie_total['Buenos Aires Interior'] = dicc_superficie_total['Palermo']
dicc_superficie_total['Villa Numancia'] = 500
dicc_superficie_total['Domselaar'] = dicc_superficie_total['San Vicente']
dicc_superficie_total['Barrio Parque San Martin'] = dicc_superficie_total['Merlo']
dicc_superficie_total['Villa Riachuelo'] = 65
dicc_superficie_total['Bs.As. G.B.A. Zona Sur'] = 333
dicc_superficie_total['Bosques'] = dicc_superficie_total['Florencio Varela']
dicc_superficie_total

{'9 de Abril': 300.0,
 'Abasto': 83.254901960784309,
 'Acacias Blancas': 334.22222222222223,
 'Acassuso': 343.41509433962267,
 'Adrogu\xc3\xa9': 232.74324324324326,
 'Agronom\xc3\xada': 89.31481481481481,
 'Albanueva Barrio Cerrado': 293.40909090909093,
 'Aldo Bonzi': 159.85714285714286,
 'Alejandro Korn': 440.0,
 'Almagro': 81.920415224913498,
 'Almirante Brown': 334.6875,
 'Altamira': 302.77777777777777,
 'Altos de Hudson II': 155.5,
 'Altos del Golf': 417.80000000000001,
 'Armenia Country Club': 206.0,
 'Arturo Segu\xc3\xad': 700.0,
 'Avellaneda': 100.60444444444444,
 'Balvanera': 95.724590163934423,
 'Banfield': 174.84615384615384,
 'Barbarita, Barrio Cerrado': 276.5625,
 'Barracas': 120.74531835205993,
 'Barrio Acacias': 383.25,
 'Barrio Cerrado "Ayres Plaza"': 277.0,
 'Barrio Cerrado "La Cautiva del Pilar"': 106.0,
 'Barrio Cerrado "La Delfina"': 271.75,
 'Barrio Cerrado "La Montura"': 277.0,
 'Barrio Cerrado "Los Omb\xc3\xbaes de Hudson"': 316.80000000000001,
 'Barrio Cerrado "L

In [88]:
superficie_covered_barrio = concatenado.loc[:, ['place_name', 'surface_covered_in_m2']]
superficie_covered_barrio = superficie_covered_barrio.groupby('place_name').agg(np.mean)
dicc_superficie_covered = {}
for x in xrange(len(superficie_covered_barrio)):
    dicc_superficie_covered[superficie_covered_barrio.index[x]] = superficie_covered_barrio.surface_covered_in_m2[x]
dicc_superficie_covered['Buenos Aires Interior'] = dicc_superficie_covered['Palermo']
dicc_superficie_covered['Villa Numancia'] = 300
dicc_superficie_covered['Domselaar'] = dicc_superficie_covered['San Vicente']
dicc_superficie_covered['Barrio Parque San Martin'] = dicc_superficie_covered['Merlo']
dicc_superficie_covered['Villa Riachuelo'] = 65
dicc_superficie_covered['Bs.As. G.B.A. Zona Sur'] = 150
dicc_superficie_covered['Bosques'] = dicc_superficie_covered['Florencio Varela']
dicc_superficie_covered

{'9 de Abril': 210.0,
 'Abasto': 86.039215686274517,
 'Acacias Blancas': 334.22222222222223,
 'Acassuso': 343.41509433962267,
 'Adrogu\xc3\xa9': 228.56081081081081,
 'Agronom\xc3\xada': 84.981481481481481,
 'Albanueva Barrio Cerrado': 293.40909090909093,
 'Aldo Bonzi': 152.71428571428572,
 'Alejandro Korn': 440.0,
 'Almagro': 80.732179930795851,
 'Almirante Brown': 247.0625,
 'Altamira': 302.77777777777777,
 'Altos de Hudson II': 155.5,
 'Altos del Golf': 417.80000000000001,
 'Armenia Country Club': 206.0,
 'Arturo Segu\xc3\xad': 90.0,
 'Avellaneda': 98.533333333333331,
 'Balvanera': 95.029508196721309,
 'Banfield': 166.79120879120879,
 'Barbarita, Barrio Cerrado': 276.5625,
 'Barracas': 118.16666666666667,
 'Barrio Acacias': 383.25,
 'Barrio Cerrado "Ayres Plaza"': 277.0,
 'Barrio Cerrado "La Cautiva del Pilar"': 106.0,
 'Barrio Cerrado "La Delfina"': 271.75,
 'Barrio Cerrado "La Montura"': 277.0,
 'Barrio Cerrado "Los Omb\xc3\xbaes de Hudson"': 316.80000000000001,
 'Barrio Cerrado "L

In [93]:
concatenado.dropna(inplace=True, subset=['lat'], how='all')
lat_barrio = concatenado.loc[:, ['place_name', 'lat']]
lat_barrio = lat_barrio.groupby('place_name').agg(np.mean)
dicc_lat = {}
for x in xrange(len(lat_barrio)):
    dicc_lat[lat_barrio.index[x]] = lat_barrio.lat[x]
dicc_lat['Buenos Aires Interior'] = dicc_lat['Palermo']
dicc_lat['Altos de Hudson I'] = dicc_lat['Altos de Hudson II']
dicc_lat['Sourigues'] = -34.805492
dicc_lat['Haras San Pablo'] = -34.6061651
dicc_lat['La horqueta de Echeverr\xc3\xada'] = -34.897642
dicc_lat['Terralagos'] = -34.907032
dicc_lat['El Roc\xc3\xado'] = -34.9070116
dicc_lat['Bs.As. G.B.A. Zona Sur'] = -34.874153
dicc_lat['Abril Club de Campo'] = dicc_lat['Berazategui']
dicc_lat

{'9 de Abril': -34.742035999999999,
 'Abasto': -34.607325200924507,
 'Abril Club de Campo': -34.786197346922187,
 'Acacias Blancas': -34.404405823977783,
 'Acassuso': -34.478191631162261,
 'Adrogu\xc3\xa9': -34.802671737867577,
 'Agronom\xc3\xada': -34.591794659698159,
 'Albanueva Barrio Cerrado': -34.41026824634546,
 'Aldo Bonzi': -34.70084856978572,
 'Alejandro Korn': -35.010582645,
 'Almagro': -34.608114592678326,
 'Almirante Brown': -34.820222345656248,
 'Altamira': -34.411126845822224,
 'Altos de Hudson I': -34.804942165919996,
 'Altos de Hudson II': -34.804942165919996,
 'Altos del Golf': -34.465240223560002,
 'Armenia Country Club': -34.430000326250003,
 'Arturo Segu\xc3\xad': -34.884135999999998,
 'Avellaneda': -34.667731659124449,
 'Balvanera': -34.609958674709461,
 'Banfield': -34.742759356510433,
 'Barbarita, Barrio Cerrado': -34.425768506099999,
 'Barracas': -34.638158766880551,
 'Barrio Acacias': -34.333242778524998,
 'Barrio Cerrado "Ayres Plaza"': -34.43722370584166,
 'B

In [94]:
lon_barrio = concatenado.loc[:, ['place_name', 'lon']]
lon_barrio = lon_barrio.groupby('place_name').agg(np.mean)
dicc_lon = {}
for x in xrange(len(lon_barrio)):
    dicc_lon[lon_barrio.index[x]] = lon_barrio.lon[x]
dicc_lon['Buenos Aires Interior'] = dicc_lon['Palermo']
dicc_lon['Altos de Hudson I'] = dicc_lon['Altos de Hudson II']
dicc_lon['Sourigues'] = -58.222173
dicc_lon['Haras San Pablo'] = -59.0397703
dicc_lon['La horqueta de Echeverr\xc3\xada'] = -58.484171
dicc_lon['Terralagos'] = -58.514642
dicc_lon['El Roc\xc3\xado'] = -58.5759269
dicc_lon['Bs.As. G.B.A. Zona Sur'] = -57.883674
dicc_lon['Abril Club de Campo'] = dicc_lon['Berazategui']
dicc_lon

{'9 de Abril': -58.489130000000003,
 'Abasto': -58.407885485899953,
 'Abril Club de Campo': -58.19269658279557,
 'Acacias Blancas': -58.762294782388899,
 'Acassuso': -58.505012415799989,
 'Adrogu\xc3\xa9': -58.39354618125607,
 'Agronom\xc3\xada': -58.485385811044488,
 'Albanueva Barrio Cerrado': -58.611577496909121,
 'Aldo Bonzi': -58.523895976600002,
 'Alejandro Korn': -58.403902379999998,
 'Almagro': -58.421056693675396,
 'Almirante Brown': -58.40545110315,
 'Altamira': -58.61506296893333,
 'Altos de Hudson I': -58.170930560090007,
 'Altos de Hudson II': -58.170930560090007,
 'Altos del Golf': -58.827255186140007,
 'Armenia Country Club': -58.793935570933336,
 'Arturo Segu\xc3\xad': -58.124928000000004,
 'Avellaneda': -58.364341783317329,
 'Balvanera': -58.403361611395447,
 'Banfield': -58.39944779521047,
 'Barbarita, Barrio Cerrado': -58.612654901968753,
 'Barracas': -58.374603455940736,
 'Barrio Acacias': -58.715020500187499,
 'Barrio Cerrado "Ayres Plaza"': -58.839529297500007,
 '

In [44]:
data2 = pd.read_csv("properati_dataset_testing_noprice.csv", low_memory = False)

In [45]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 17 columns):
id                         14166 non-null int64
created_on                 14166 non-null object
property_type              14166 non-null object
operation                  14166 non-null object
place_name                 14166 non-null object
place_with_parent_names    14166 non-null object
country_name               14166 non-null object
state_name                 14166 non-null object
lat-lon                    10487 non-null object
lat                        10487 non-null float64
lon                        10487 non-null float64
surface_total_in_m2        11853 non-null float64
surface_covered_in_m2      13005 non-null float64
floor                      1368 non-null float64
rooms                      7500 non-null float64
expenses                   2543 non-null object
description                14166 non-null object
dtypes: float64(6), int64(1), object(10)
memory usage: 1

In [98]:
data_completa = data2

In [113]:
def llenadoDeSuperficieTotal(reg):
    
    if reg['surface_total_in_m2'] == 0 or np.isnan(reg['surface_total_in_m2']):
        reg['surface_total_in_m2'] = dicc_superficie_total[reg['place_name']]
    return reg['surface_total_in_m2']

def llenadoDeSuperficieCovered(reg):
    
    if reg['surface_covered_in_m2'] == 0 or np.isnan(reg['surface_covered_in_m2']):
        reg['surface_covered_in_m2'] = dicc_superficie_covered[reg['place_name']]
    return reg['surface_covered_in_m2']

def llenadoDeLatitud(reg):
    
    if np.isnan(reg['lat']):
        reg['lat'] = dicc_lat[reg['place_name']] 
    return reg['lat']

def llenadoDeLongitud(reg):
    
    if np.isnan(reg['lon']):
        reg['lon'] = dicc_lat[reg['place_name']] 
    return reg['lon']

In [121]:
data_completa = data_completa.loc[:,['created_on', 'property_type', 'place_name', 'lat', 'lon', 'surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms', 'description']]
data_completa['surface_covered_in_m2'] = np.where(np.isnan(data_completa['surface_covered_in_m2']), data_completa['surface_total_in_m2'], data_completa['surface_covered_in_m2'])
data_completa['surface_total_in_m2'] = np.where(np.isnan(data_completa['surface_total_in_m2']), data_completa['surface_covered_in_m2'], data_completa['surface_total_in_m2'])
data_completa['lat'] = data_completa.apply(llenadoDeLatitud, axis=1)
data_completa['lon'] = data_completa.apply(llenadoDeLongitud, axis=1)
data_completa['surface_total_in_m2'] = data_completa.apply(llenadoDeSuperficieTotal, axis=1)
data_completa['surface_covered_in_m2'] = data_completa.apply(llenadoDeSuperficieCovered, axis=1)
data_completa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 10 columns):
created_on               14166 non-null object
property_type            14166 non-null object
place_name               14166 non-null object
lat                      14166 non-null float64
lon                      14166 non-null float64
surface_total_in_m2      14166 non-null float64
surface_covered_in_m2    14166 non-null float64
floor                    1368 non-null float64
rooms                    7500 non-null float64
description              14166 non-null object
dtypes: float64(6), object(4)
memory usage: 1.1+ MB


,created_on,property_type,place_name,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,description
0,2017-08-24,departamento,Puerto Madero,-34.610988,-58.363464,153.232402,152.560894,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...
1,2017-08-25,departamento,Buenos Aires Interior,-34.583264,-34.583264,102.525953,99.639201,NaN,NaN,El departamento cuenta con un living-comedor a...
2,2017-08-01,departamento,Palermo Soho,-34.589363,-58.412880,53.000000,48.000000,NaN,NaN,IMPECABLE TORRE COY III – DEPA...
3,2017-08-01,departamento,Chacarita,-34.587703,-34.587703,39.000000,39.000000,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
4,2017-08-01,departamento,Chacarita,-34.587703,-34.587703,51.000000,51.000000,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...
5,2017-08-01,departamento,Chacarita,-34.587703,-34.587703,39.000000,39.000000,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
6,2017-08-01,departamento,Chacarita,-34.587703,-34.587703,39.000000,39.000000,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
7,2017-08-01,departamento,Chacarita,-34.587703,-34.587703,39.000000,39.000000,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
8,2017-08-01,departamento,Colegiales,-34.578721,-58.457095,67.000000,67.000000,NaN,NaN,EXCELENTE 2 AMB- APTO PROFESIO...
9,2017-08-01,departamento,Almagro,-34.608115,-34.608115,47.000000,47.000000,NaN,NaN,IMPECABLE MONOAMBIENTE- DIVISI...
